**Enrichment Homework 3b:**

    COVID-19 Data Restructuring
Rashid Baembitov, Your UIN, 689

**Motivation/Importance:**

Existing Problem:

Ongoing pandemic of corona virus. The topic is important, because people are dying. It is interesting topic, because it may save lives.

Potential Impact:

Impact is that we may better understand and summarize different practices used to cope with pandemic. That allows one to choose best methods and be prepared for possible future pandemics. Policymakers from aroung the globe can assess practices from another countries and adjust them to their situation

The authors of [1] made an early publication, when the spread of corona virus was mainly concentrated in China. The article was published in March, based on situation in China. Authors predicted that the situation similar to theirs may occure in other countries all over the world. Well, they were right... Authors claimed that measures on the virus from the government would help to curtail the spread. As soon as pandemic has spread, the World Health Organisation released a scientific report on the Off-label use of medicines for COVID-19 [2]. It appears that "No pharmaceutical products have yet been shown to be safe and effective for the treatment of COVID-19" as the report states.

There were different measures taken to slow down the spread of the virus and allow healthcare organisations prepare to fight the virus. The article [3] analazes the real-time mobility data from Wuhan and detailed case data including travel history to estimate the role of case importation in transmission in cities across China and to ascertain the impact of control measures. Authors claim that in the beginning, the spatial distribution of COVID-19 cases in China was explained well by human mobility data. After imposing of control measures, this correlation dropped and growth rates became negative in most locations, although shifts in the demographics of reported cases were still indicative of local chains of transmission outside of Wuhan.

New COVID-19 virus belongs to the same family of viruses as the Severe Acute Respiratory Syndrome or SARS virus, outbreak of which was registered in Guangdong province of southern China in 2002. The authors of [4] conduct a thorough research on the similarities of the two types of viruses in the context of their virus incubation, originations, diagnosis and treatment methods, genomic and proteomic sequences, and pathogenic mechanisms. Their results showed that almost all encoded proteins of SARS-CoV-2 are homologous to SARS-CoV proteins. Another group of authors arrive into same conclusion, as can be understood from [5].

A question arises: if the measures, used to cope with the spread of SARS virus in 2002 in Guangdong, can be applied and be effective towards the new COVID-19 and thus reduce the peak of the cases? The authors of the paper [6] are trying to adress that question. They report that traditional public health measures used during SARS were successful and included active case detection, isolation of cases, contact tracing and quarantine of all contacts, social distancing, and community quarantine. However, the authors arrive to the conclusion that even though two viruses are very alike, the same measures might not be enough to eradicate the pandemic. At the same time, they deem the constraints imposed by governments on people movement and other containing strategies to be neccessary to slow down the peak cases and thus very appropriate for the current situation.

In [7] authors reviewed the existing litrature on the SARS family viruses and suggeted special measures on diagnosis and potential interventions. Among other measures researchers highlight isolation and early detection and diagnosis along with the tracking of the potential contacts of a patient. This conclusion is supported by the work [8], where authors conducted a simulation study on effects of intervention measures of the COVID-19. Authors specifically mention the enormous burden on the healthcare institutions and recommend the strict control measures help to contain disease from spreading and an early detecting, reporting and fast reacting to control the spread.

The dataset being analyzed is one of the datasets available on Kaggle platform. As will be seen later, it includes some qualitative features that need to be encoded for the future use in ML models. One of the methods used is "one hot encoding" to convert qualitative data into quantative. Authors of [9] used the same technique in one of the steps of their framework for diagnosing COVID-19 in X-Ray Images using deep learning classifiers, called COVIDX-Net.

Another technique implemented on that dataset is Term Frequency - Inverse Document Frequency technique. It is applied to the "comments" part of the dataset. The same approach was used in [10] to understand emotional responses to COVID-19. Authors conclude that emotional responses correlated with linguistic measures.

## Methodology

Data Source

The dataset is aquired from Kaggle competition on COVID-19.

Restructuring Details

As we move through the notebook we will adress different columns, discuss their contents and use different restructuring techniques.


In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Goverment measures dataset

In [ ]:
data = pd.read_csv('/kaggle/input/uncover/UNCOVER/HDE/acaps-covid-19-government-measures-dataset.csv')
data.head()

## Check for missing values

Before we proceed lets see how much data is missing from each column.
We will use that information later to fill missing values if needed

In [ ]:
data.isnull().sum()

## Reported measures by country

First we will look which countries are present in dataset and count their occurences

In [ ]:
countries = data['country'].value_counts()
countries

So there are 193 different countries present with different amount of measures reported.

We will restructure the acquired data into dataframe with country name and number of mentions in the original dataset.
Then we will plot the countries on the map to see visual representation of the dataset

In [ ]:
countries = countries.to_frame().reset_index().rename(columns={"index": "Country", "country": "mentions"})
countries.head()

Plotting using plotly

In [ ]:
import plotly.express as px

fig = px.choropleth(countries, locations="Country", locationmode='country names', 
                  color="mentions", hover_name="Country", 
                  title="mentions of governmetn measures", hover_data=["mentions"])
fig.show()

Now we will look into the category columns and assess the data in it

In [ ]:
data['category'].value_counts()

lets plot that using pie chart for better visualization

In [ ]:
fig = px.pie(data['category'].value_counts(), 
             values = data['category'].value_counts(), 
             names=data['category'].value_counts().index,
            title = "Categories of the measures")
fig.show()

## One Hot Encoding

In order to use categorical values in ML models, categorical values need to be converted into numerical. In other words, we need to convert qualitative data into quantative. One of the methods is One Hot Encoding, which creates additional columns in the quantaty of numner of unique categorical values and puts "1" into the columns of the corresponding feature.**

In [ ]:
dummies = pd.get_dummies(data['category'])
dummies.head()

We connect the tranasformed data to original dataset

In [ ]:
data = pd.concat([data, dummies], axis=1, sort=False)
data.head()

## Label Encoding

Another column that needs restucturing is the "measure" column. However, as will be seen further, it contains too many types of measures, thus we would use another method - Label Encoding. It is essentially assigning a number to each category.



In [ ]:
stat = data['measure'].value_counts()
stat.head()

In [ ]:
fig = px.pie(stat, 
             values = stat, 
             names=stat.index,
            title = "Measures")
fig.show()

First we would convert string type to category type. It will make it easier to perform label encoding

In [ ]:
data['measure'] = data['measure'].astype('category')
data.dtypes

Now we will use *cat.codes* to get numerical value for each category and put it into original dataset

In [ ]:
data['measure_cat'] = data['measure'].cat.codes
data.head()

> ## TF IDF and Bag of Words

We now mooving to the comments column. Which is a text data and it is hard to restructure it in a way, so we can gather some information out of it. One of the ways we learned in class is Grounded Theory. Alas it takes a lot of manual work and usually would require several people to get rid of possible bias. 
Here we will try another methods

Here is how many different comments we have

In [ ]:
len(data['comments'].value_counts())

Recall that some of the "comments" cells are empty, thus we would just put an empty string in those cells. Otherwise some methods do not work properly. Putting blanks is reasonable way, since we do not alter the data in any way

In [ ]:
data['comments'].fillna('',inplace=True)

To further restructure our data we would delete any punctuation and also convert everything to lower case

In [ ]:
data['comments_modified'] = data['comments'].str.strip('.,;:""')
data['comments_modified'] = data['comments_modified'].str.lower()

We also would remove all numerical characters

In [ ]:
data['comments_modified'] = data['comments_modified'].str.translate({ord(k): None for k in '0123456789'})

In [ ]:
data['comments_modified'].head()

Importing necessary tools

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from itertools import islice

We define an instance of the CountVectorizer class. 

We would use english stop words list to get rid of the words that should not be accounted as important words.
min_df is the parameter that specifies minimum document frequency for the term to be accounted for. We set in to 2.
max_df parameter similarly filters words that are present in more that the specified percentage of the rows analyzed.
nrgam_range specifies the lenght of the ngrams to be considered. Here we would just use the single words.

In [ ]:
vectorizer = CountVectorizer(stop_words='english', min_df=2, max_df=0.9, ngram_range=(1,1))
vectorizer

Calling vectorizer on our dataset of comments

In [ ]:
vectorizer.fit(data['comments_modified'])

We can look at the part of the constructed vocablurary:

In [ ]:
list(islice(vectorizer.vocabulary_.items(), 20))

The total different words we have accounted for is:

In [ ]:
len(vectorizer.vocabulary_)

Now we will transform our text into so called "bag of words", which is essentially separate column for each word containing the count within each row.

As can be seen from code below the resulting matrix is really sparce

In [ ]:
bag_of_words = vectorizer.transform(data['comments_modified'])
print( "shape ", bag_of_words.toarray().shape)
print('sparsity: %.2f%%' % (100.0 * bag_of_words.nnz / (bag_of_words.shape[0] * bag_of_words.shape[1])))

We can look at the most encountered words in the data:

In [ ]:
occ = np.asarray(bag_of_words.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(20)

Now we will transform the "bag of words" matrix into tf-idf matrix.

tf-idf means term-frequency times inverse document-frequency. This is a common term weighting scheme in information retrieval, that has also found good use in document classification.

The formula that is used to compute the tf-idf for a term t of a document d in a document set is tf-idf(t, d) = tf(t, d) * idf(t), and the idf is computed as idf(t) = log [ n / df(t) ] + 1, where n is the total number of documents in the document set and df(t) is the document frequency of t; the document frequency is the number of documents in the document set that contain the term t. The effect of adding “1” to the idf in the equation above is that terms with zero idf, i.e., terms that occur in all documents in a training set, will not be entirely ignored.

In [ ]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(bag_of_words)
transformed_weights.shape

Essentially we normed the "bag of words" matrix by the inverse document frequency

Now we can look at the most important, judging by their mean

In [ ]:
importance = transformed_weights.toarray().mean(axis=0)
im_df = pd.DataFrame({'word': vectorizer.get_feature_names(), 'importance': importance})
im_df.sort_values(by='importance', ascending=False).head(15)

Thus we transformed the textual data into the matrix, that contains information in numerical form.
Of course, the TF-IDF matrix is not perfect, but it is some way of analyzing big textual data.
The TF-IDF matrix now can be used as an input into some algorithm.

## n_grams

We would try to further mine the dataset for useful information.

As was noted in the previous section, the TF-IDF matrix is sparce, which is not always good for algorithms.
We would try to first extract important 2_grams and 3_grams using TF-IDF and then create a representation of original dataset, that would indicate if the row has the important n_gram or not. Basically, we would encode a feature that shows if the row has an expression of intrest or not.

We would use the TfidfVectorizer class that unites CountVectorizer and TfidfTransformer in one class.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer1 = TfidfVectorizer(stop_words='english', min_df=2, max_df=0.9, ngram_range=(2,3))
weights_n_gram = vectorizer1.fit_transform(data['comments_modified'])
importance_n_gram = weights_n_gram.toarray().mean(axis=0)
im_df_n_gram = pd.DataFrame({'word': vectorizer1.get_feature_names(), 'importance': importance_n_gram})
im_df_n_gram.sort_values(by='importance', ascending=False).head(20)

Out of first 20 n_grams we would select the ones that make most sence for a human (this is strictly subjective procedure, opinions may differ):

In [ ]:
features = ['state of emergency', 
            'non essential', 
            'public gatherings', 
            'self quarantine',
            'international flights',
            'stay home',
            'schools closed ',
            'social distancing'] #we added some stop words and modifications back to the text, so we can find them in original dataset

Now we would analyze "comments" column and create corresponding features

In [ ]:
for feature in features:
    data[feature] = data['comments_modified'].apply(lambda x: 1 if feature in x else 0)

In [ ]:
data.head()

The folliwing code shows, how much rows have the features of interest:

In [ ]:
for feature in features:
    print(feature, ' is in ', data[feature].value_counts()[1], ' rows')
    

References

1. Roy M Anderson, Hans Heesterbeek, Don Klinkenberg, T Déirdre Hollingsworth, “How will country-based mitigation measures influence the course of the COVID-19 epidemic?“, Lancet Infectious Diseases, vol. 395, pp. 931-934, Mar. 2020.
1. World Health Organization. (31 March 2020 ). Scientific brief: Off-label use of medicines for COVID-19 ( ed.) [Online]. Available: https://www.who.int/news-room/commentaries/detail/off-label-use-of-medicines-for-covid-19
1. Moritz U. G. Kraemer, Chia-Hung Yang, Bernardo Gutierrez, Chieh-Hsi Wu, Brennan Klein, David M. Pigott et al., “The effect of human mobility and control measures on the COVID-19 epidemic in China“, Science, vol. 368, pp. 493-497, 01 May 2020.
1. Xu, Jiabao et al. “Systematic Comparison of Two Animal-to-Human Transmitted Human Coronaviruses: SARS-CoV-2 and SARS-CoV.” Viruses vol. 12,2 244. 22 Feb. 2020, doi:10.3390/v12020244
1. Chan J.F.W., Kok K.H., Zhu Z., Chu H., To K.K.W., Yuan S., Yuen K.Y. "Genomic characterization of the 2019 novel human-pathogenic coronavirus isolated from a patient with atypical pneumonia after visiting Wuhan", Emerg. Microbes. Infect. 2020;9:221–236. doi: 10.1080/22221751.2020.1719902.
6. Annelies Wilder-Smith, Calvin J Chiew, Vernon J Lee, “Can we contain the COVID-19 outbreak with the same measures as for SARS?“, The Lancet Infectious Diseases, vol. 20, pp. e102-e107, 5, May 2020.
7. Wang, Y, Wang, Y, Chen, Y, Qin, Q. "Unique epidemiological and clinical features of the emerging 2019 novel coronavirus pneumonia (COVID‐19) implicate special control measures." J Med Virol. 2020; 92: 568– 576. https://doi.org/10.1002/jmv.25748
1. Zhou G, Chi C. "A model simulation study on effects of intervention measures in Wuhan COVID‐19 epidemic." med Rxiv. 2020. https://doi.org/10.1101/2020.02.14.20023168
1. zz El-Din Hemdan, Marwa A. Shouman, Mohamed Esmail Karar, “COVIDX-Net: A Framework of Deep Learning Classifiers to Diagnose COVID-19 in X-Ray Images“, arXiv:2003.11055v1 [eess.IV], 24 Mar 2020.
1. Bennett Kleinberg, Isabelle van der Vegt, Maximilian Mozes, “Measuring Emotions in the COVID-19 Real World Worry Dataset“, arXiv:2004.04225 [cs.CL], 14 May 2020.

POST SCRIPTUM.

Special thanks goes to real MVPs of this world, who show how to use the tools:

http://www.ultravioletanalytics.com/blog/tf-idf-basics-with-pandas-scikit-learn

https://stackoverflow.com/questions/37593293/what-is-the-simplest-way-to-get-tfidf-with-pandas-dataframe

https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089

https://chrisalbon.com/machine_learning/preprocessing_text/bag_of_words/

https://stackabuse.com/python-for-nlp-creating-bag-of-words-model-from-scratch/

https://towardsdatascience.com/gentle-start-to-natural-language-processing-using-python-6e46c07addf3

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

https://chrisalbon.com/machine_learning/preprocessing_text/tf-idf/